<a href="https://colab.research.google.com/github/ankitaggarwal64/Agricultural-Equipment-Sales-Forecasting/blob/main/Project_Overview_and_Data_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Agriculture Equipment Monthly Sales Forecasting** 
	
**Overview:**
This project is done to provide a multinational automobile company with the best sales forecasting model for their agricultual equipments. Along with the actual sales data, company provided other relevant data including economic and commodity indices which might be helpful for forecasting.



---


**Data Description:**
Following two types of information is provided :
1.   *Past Sales Data :* January 1990 to December 2016 (27 Years)


2.  *Additional Predictive Information ( 3 excel worksheets)*
*   Monthly Macroeconomic Indices ( GDP, Unemployment rate, CPI, 30YearMtggRate etc.)
*   Monthly Commodity Indices ( Hay, Corn, Wheat, Dairy, Livestock, BRLCSPOT.D7 etc.)
*    Annual Indices for US agricultural and economic forecast information 



---



**Problem Statement:**
Develop an accurate sales prediction model for forecasting the sales for next 12 months in future.


---
**Metrics**:

 

*   RMSE(Root Mean Squared Error) will be considered as primary performance metric.
*   Other tracked metrics: MAPE(Mean Absolute Percentage) and Bias
*   Uncertainity in forecasting will also needs to be closed tracked.






**Data Understanding and Data Ingestion :**
 Further in this noteook, we will look at all the provided data, filter out the relevant data and try to consolidate all the required data in a single file.

In [23]:
# Importing libraries 
import pandas as pd

In [24]:
# Read sales data excel workbook
sales_workbook = pd.ExcelFile("/content/drive/My Drive/1. Data Science/Sales Forecasting_Masters Project/Masters Project/Data/Raw Data/AG Industry_2017-02-13.xlsx")
# Listing out the worksheet in the excel workbook
print(sales_workbook.sheet_names) 
# Looks like each sheet correspond to sales data for a particular product

['Total', 'Tractors 0-20', 'Tractors 20-40', 'Tractors 0-40', 'Tractors 40-60', 'Tractors 60-100', 'Tractors 100-140', 'Tractors 140+', '4WD', 'Combines', 'SP Sprayers', 'DMC', 'SMC', 'RND B', 'SM SQ B', 'LG SQ  B', 'SPFH ', 'SPW']


In [25]:
# Creating a dictionary of sheets in workbook
Sales_dict = {sheet: sales_workbook.parse(sheet) for sheet in sales_workbook.sheet_names}
print(type(Sales))

<class 'pandas.core.frame.DataFrame'>


As a proof of concept, we will first work on only one product "4wd" from all the available products data 


In [26]:
# Since we are interested in "4wd" sales data, let's extract the corresponding worksheet
Sales = Sales_dict['4WD']
print(Sales)
print(type(Sales))
print(Sales.shape)

    INDUSTRY  JAN  FEB  MAR   APR   MAY  ...  AUG  SEP   OCT  NOV  DEC  TOTAL
0       1990  505  467  467   593   866  ...  371  269   778  618  531   6771
1       1991  241  231  506   703   765  ...  319  296   780  247  522   5483
2       1992  271  209  484   413   342  ...  179  168   498  342  323   3719
3       1993  313  228  367   613   358  ...  231  277   555  645  698   4839
4       1994  356  247  594   574   364  ...  218  268   582  473  550   4781
5       1995  366  452  655   810   507  ...  265  316   749  620  706   5920
6       1996  457  485  591   977   515  ...  229  380   839  486  564   6169
7       1997  378  485  759  1272  1044  ...  451  557   968  654  711   8337
8       1998  488  434  759   878   468  ...  217  227   500  325  372   5329
9       1999  335  308  395   571   367  ...  133  232   421  257  324   3804
10      2000  223  236  420   590   294  ...  187  211   466  144  268   3529
11      2001  255  244  491   732   385  ...  158  169   430  22

Observations from Sales data:  
1.   Monthly sales are provided from 1990 to 2016 (27 Years)

2.   There are no missing values 

In [27]:
# Checking Duplicates
Sales.duplicated().sum() 
# No duplicate rows are present

0

Now we would like to sales data dataframe to a single column time series format 


In [33]:
# Removing the yearly total sales column as this is not required and can be generated when ever requried in analysis
if "TOTAL" in Sales.columns:
  Sales = Sales.drop("TOTAL", axis=1)

In [34]:
Sales.shape
display(Sales)

,INDUSTRY,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1990,505,467,467,593,866,715,591,371,269,778,618,531
1,1991,241,231,506,703,765,576,297,319,296,780,247,522
2,1992,271,209,484,413,342,310,180,179,168,498,342,323
3,1993,313,228,367,613,358,346,208,231,277,555,645,698
4,1994,356,247,594,574,364,296,259,218,268,582,473,550
5,1995,366,452,655,810,507,258,216,265,316,749,620,706
6,1996,457,485,591,977,515,430,216,229,380,839,486,564
7,1997,378,485,759,1272,1044,584,474,451,557,968,654,711
8,1998,488,434,759,878,468,315,346,217,227,500,325,372
9,1999,335,308,395,571,367,266,195,133,232,421,257,324


In [35]:
import numpy as np
Sales_t = Sales.T
display(Sales_t)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
INDUSTRY,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
JAN,505,241,271,313,356,366,457,378,488,335,223,255,271,215,255,292,251,230,273,313,327,506,359,678,631,220,243
FEB,467,231,209,228,247,452,485,485,434,308,236,244,184,175,206,171,264,188,285,382,456,461,487,669,575,346,222
MAR,467,506,484,367,594,655,591,759,759,395,420,491,374,335,448,381,446,449,585,499,705,624,745,702,655,499,258
APR,593,703,413,613,574,810,977,1272,878,571,590,732,562,642,704,833,487,678,610,608,853,883,883,851,745,430,335
MAY,866,765,342,358,364,507,515,1044,468,367,294,385,352,341,290,351,300,394,512,513,461,588,611,554,474,260,234
JUN,715,576,310,346,296,258,430,584,315,266,226,281,233,177,344,308,274,266,435,436,569,426,499,539,408,300,247
JUL,591,297,180,208,259,216,216,474,346,195,264,244,134,176,244,240,230,235,347,448,435,457,552,491,389,268,152
AUG,371,319,179,231,218,265,229,451,217,133,187,158,112,128,171,243,151,167,412,399,412,434,583,587,369,167,102
SEP,269,296,168,277,268,316,380,557,227,232,211,169,172,256,202,286,240,302,545,559,653,700,774,571,486,249,200


In [36]:
Sales_t.columns= Sales_t.iloc[0]
Sales_tt = Sales_t.drop("INDUSTRY",axis=0)
display(Sales_tt)

INDUSTRY,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
JAN,505,241,271,313,356,366,457,378,488,335,223,255,271,215,255,292,251,230,273,313,327,506,359,678,631,220,243
FEB,467,231,209,228,247,452,485,485,434,308,236,244,184,175,206,171,264,188,285,382,456,461,487,669,575,346,222
MAR,467,506,484,367,594,655,591,759,759,395,420,491,374,335,448,381,446,449,585,499,705,624,745,702,655,499,258
APR,593,703,413,613,574,810,977,1272,878,571,590,732,562,642,704,833,487,678,610,608,853,883,883,851,745,430,335
MAY,866,765,342,358,364,507,515,1044,468,367,294,385,352,341,290,351,300,394,512,513,461,588,611,554,474,260,234
JUN,715,576,310,346,296,258,430,584,315,266,226,281,233,177,344,308,274,266,435,436,569,426,499,539,408,300,247
JUL,591,297,180,208,259,216,216,474,346,195,264,244,134,176,244,240,230,235,347,448,435,457,552,491,389,268,152
AUG,371,319,179,231,218,265,229,451,217,133,187,158,112,128,171,243,151,167,412,399,412,434,583,587,369,167,102
SEP,269,296,168,277,268,316,380,557,227,232,211,169,172,256,202,286,240,302,545,559,653,700,774,571,486,249,200
OCT,778,780,498,555,582,749,839,968,500,421,466,430,454,571,780,612,512,809,734,687,1117,1125,1498,1225,731,515,611


In [39]:
Sales_melt = Sales_tt.melt(id_vars=None)
Sales_melt.columns = ["Year","Sales"]
display(Sales_melt)

,Year,Sales
0,1990,505
1,1990,467
2,1990,467
3,1990,593
4,1990,866
...,...,...
319,2016,102
320,2016,200
321,2016,611
322,2016,205


In [55]:
# Creating a dataframe of date range for using it in final Sales table
Time = pd.DataFrame(pd.date_range(start='1990',end = "2017", freq='M'))

,0
0,1990-01-31
1,1990-02-28
2,1990-03-31
3,1990-04-30
4,1990-05-31
...,...
319,2016-08-31
320,2016-09-30
321,2016-10-31
322,2016-11-30


In [63]:
#Creating a Final Sales table in Time series format
Sales_ts = pd.concat([Time,Sales_melt["Sales"]],axis=1)
Sales_ts.columns = ["Time","Sales"]
Sales_ts = Sales_ts.set_index("Time")
Sales_ts

,Sales
Time,
1990-01-31,505
1990-02-28,467
1990-03-31,467
1990-04-30,593
1990-05-31,866
...,...
2016-08-31,102
2016-09-30,200
2016-10-31,611
